## TwoD Cases

In [ ]:
import os, sys
import numpy as np
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.TwoDSubduction0.Cases as TwoDCases
import shilofue.PlotCase as PlotCase
import shilofue.TwoDSubduction0.PlotCase as TwoDPlotCase
import shilofue.TwoDSubduction0.VtkPp as TwoDVtkPp
import shilofue.PlotDepthAverage as PlotDepthAverage
from shilofue.PlotCombine import PlotCombineExecute, PlotCombineFigures

#### System path

The path to the project directory

    local_TwoDSubduction_dir

The path to the project directory on server

    remote_TwoDSubduction_dir

In [ ]:
local_TwoDSubduction_dir = "/mnt/lochy0/ASPECT_DATA/TwoDSubduction"
remote_TwoDSubduction_dir = "peloton:/group/billengrp-mpi-io/lochy/TwoDSubduction"

assert(os.path.isdir(local_TwoDSubduction_dir))

#### case name

case name (relative path to local_TwoDSubduction_dir)

    case_name

In [ ]:
case_name = "EBA_CDPT_10/eba_cdpt_SA80.0_OA40.0_gr10_new"
local_dir = os.path.join(local_TwoDSubduction_dir, case_name)
remote_dir = os.path.join(remote_TwoDSubduction_dir, case_name)

#### rsync command

Print the rsync command which could run in a terminal

In [ ]:
rsync_exec = "rsync"
rsync_option = "-auv --progress"
exclude_option = "--exclude=*restart*"
local_path = os.path.join(local_TwoDSubduction_dir, os.path.dirname(case_name)) + "/"
remote_path = remote_dir
paths = remote_path + " " + local_path

rsync_command = "%s %s %s %s %s" % (rsync_exec, rsync_option, exclude_option, remote_path, local_path)

print("open a terminal and run rsync command:\n", rsync_command) 

#### Plot Linear results and prepare the scripts for visualization

The next block will generate plots of run time, solver, statistics, etc.
It will also compose a script to run in paraview / visit.

It would be prefered to first run the next block for slab morphology, analyze the result and fix the steps to visualize in this block.

The max_velocity is useful for rescaling the color for the velocity. To pick up a reasonable value, first view the range from outputs with
max_velocity equals -1.0.
The unit to use is m / yr.


In [ ]:
# turn on plot_axis if I want to save a complete result
# turn off if I want to prepare for figures in a paper
plot_axis = True
step = [0, 100]
max_velocity = -1.0  # rescale the color for velocity
# visual_type = "paraview"
visual_type = "pygmt"

PlotCase.PlotCaseRun(local_dir, time_range=None, run_visual=False,\
        time_interval=None, step=step)
plt.close() # plot won't show up below

TwoDPlotCase.PlotCaseRun(local_dir, time_range=None, run_visual=False,\
        time_interval=None, visualization=visual_type, step=step, plot_axis=plot_axis, max_velocity=max_velocity)
plt.close() # plot won't show up below

#### Plot the horizontal average profile

A time t (in yr) is assigned to plot horizontal average at that time.

In [ ]:
plot_time = 0

depth_average_path = os.path.join(local_dir, 'output', 'depth_average.txt')
assert(os.path.isfile(depth_average_path))

fig_path_base = os.path.join(local_dir, 'img', 'DepthAverage.png')
PlotDepthAverage.PlotDaFigure(depth_average_path, fig_path_base, time=plot_time)

### Visualization, pt 1 : run paraview

run the following command with the script generated in the last block:

    paraview --script {paraview scripts}

or run without gui: 

    pvpython {paraview scripts}


#### adjust camera, run the following in Tools -> Python Script Editor

1. Upper mantle view:

```

# get active view
#### import the simple module from the paraview
from paraview.simple import *
#### disable automatic camera reset on 'Show'
paraview.simple._DisableFirstRenderCameraReset()

# get active view
renderView1 = GetActiveViewOrCreate('RenderView')

#-----------------------------------
# saving camera placements for views

# current camera placement for renderView1
renderView1.InteractionMode = '2D'
renderView1.CameraPosition = [0.0, 5.6e5, 2.5e7]
renderView1.CameraFocalPoint = [0.0, 6e6, 0.0]
renderView1.CameraParallelScale = 4.5e5

```

### Plot Slab Morphology

First analyze slab morphology (first cell).
Then plot slab morphology (second cell).

Interval to analyze.
0.5e6 is tested to give the best results in terms of capturing the trending and smearing out minor spikes.

    time_interval

In [ ]:
time_interval = 0.5e6

TwoDVtkPp.SlabMorphologyCase(local_dir, rewrite=1, findmdd=True, time_interval=time_interval, project_velocity=True)

In [ ]:
SlabPlot = TwoDVtkPp.SLABPLOT('slab')
SlabPlot.PlotMorph(local_dir, save_pdf=True)

plt.close()

### Pick Timesteps

Pick in WebPlotDigitizer the time steps of:

1. Slab tip reaching 660 ($t_{660}$).
2. First peak of trench motion after $t_{660}$ ($t_{p1}$), and trench velocity
3. Second peak of trench motion after $t_{660}$ ($t_{p2}$), and trench velocity

### Visualization, pt 2 : save plots at these steps

Navigate to the paraview_scripts/slab.py file and change the steps.

### Plot Morphology Extra : Compare Morphology

Compare the slab morphology from multiple cases.

In [ ]:
case_name1 = "EBA_CDPT9/eba_cdpt_SA80.0_OA40.0_pc_mei_gr10_cf25GPa_PT1800_1"

json_option =\
{
    "_comment" : "This is configuration for combining results of time run (time, step, dofs ...)",
    "case_root": local_TwoDSubduction_dir,
    "cases": [case_name, case_name1],
    "output directory": {
        "relative": 1,
        "path": "plot_combine"
    },
    "width": -1.0,
    "time range": [0.0, 60e6],
    "trench position range": [-200e3, 200e3],
    "slab depth range": [0e3, 1800e3]
}

# print("json_option: ", json_option) # debug
PlotCombineExecute(TwoDVtkPp.PLOT_COMBINE_SLAB_MORPH, TwoDVtkPp.PC_MORPH_OPT, "slab_morph", json_option)

### Compare between case outputs

In [ ]:
json_option = \
{
	"case_root": "${TwoDSubduction_DIR}",
    "Name": "combine_figure",
	"cases": [
        "EBA_CDPT4/eba_cdpt_SA80.0_OA40.0_CpEcl_1",
        "EBA_CDPT3/eba_cdpt_SA80.0_OA40.0_width140"
    ],
	"plots": [
        ["um_viscosity_snap000005.png", "um_viscosity_snap000185.png", "um_viscosity_snap000305.png"],
        ["um_viscosity_snap000005.png", "um_viscosity_snap000125.png", "um_viscosity_snap000225.png"]
    ],
    "anchor": 0,
	"same_for_all" : 0,
    "output directory": "${TwoDSubduction_DIR}/EBA_CDPT4",
    "Include title": 1,
    "Title": {
        "Title": "Comparison of implementation of the eclogite transition, the lowP case has a lowP boundary with a clayperon slope."
    }
}

PlotCombineFigures(json_option)